<a href="https://colab.research.google.com/github/hurricane195/Intro-to-Deep-Learning/blob/Homework_4/HW4_P3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 3B (Redo of Problem 2)**

Repeat problems 1 and 2, this time trying to translate from French to English. Train the model on the entire dataset and evaluate it on the entire dataset. Report training loss, validation loss, and validation accuracy. Also, try some qualitative validation, asking the network to generate English translations for some French sentences. Which one is more effective, French-to-English or English-to-French?

In [ ]:
#Using a modided example of Dr. Tabkhi's "sequence2sequence" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/sequence2sequence.py
#Using a modided example of Dr. Tabkhi's "E2F-loader" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/E2F-loader.py
#Using a modided example of Dr. Tabkhi's "attention" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/attention.py
#Random help from Chat GPT on formatting, sytntax, etc.
#Random help from Chat Colab AI on formatting, sytntax, etc.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Vocabulary:
    def __init__(self):
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.word_count = {}
        self.n_words = 3  # Start counting from 3 to account for special tokens

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            self.word_count[word] += 1

def tokenize_and_pad(sentences, vocab):
    max_length = max(len(sentence.split(' ')) for sentence in sentences) + 2  # For SOS and EOS
    tokenized_sentences = []
    for sentence in sentences:
        tokens = [vocab.word2index["<SOS>"]] + [vocab.word2index.get(word, vocab.word2index["<PAD>"]) for word in sentence.split(' ')] + [vocab.word2index["<EOS>"]]
        padded_tokens = tokens + [vocab.word2index["<PAD>"]] * (max_length - len(tokens))
        tokenized_sentences.append(padded_tokens)
    return torch.tensor(tokenized_sentences, dtype=torch.long)

In [ ]:
class FrEngDataset(Dataset):
    def __init__(self, pairs):
        self.fr_vocab = Vocabulary()
        self.eng_vocab = Vocabulary()
        self.pairs = []

        for fr, eng in pairs:
            self.fr_vocab.add_sentence(fr)
            self.eng_vocab.add_sentence(eng)
            self.pairs.append((fr, eng))

        self.fr_tokens = tokenize_and_pad([fr for fr, _ in pairs], self.fr_vocab)
        self.eng_tokens = tokenize_and_pad([eng for _, eng in pairs], self.eng_vocab)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.fr_tokens[idx], self.eng_tokens[idx]

In [ ]:
french_to_english = [
    ("J'ai froid", "I am cold"),
    ("Tu es fatigué", "You are tired"),
    ("Il a faim", "He is hungry"),
    ("Elle est heureuse", "She is happy"),
    ("Nous sommes amis", "We are friends"),
    ("Ils sont étudiants", "They are students"),
    ("Le chat dort", "The cat is sleeping"),
    ("Le soleil brille", "The sun is shining"),
    ("Nous aimons la musique", "We love music"),
    ("Elle parle français couramment", "She speaks French fluently"),
    ("Il aime lire des livres", "He enjoys reading books"),
    ("Ils jouent au football chaque week-end", "They play soccer every weekend"),
    ("Le film commence à 19 heures", "The movie starts at 7 PM"),
    ("Elle porte une robe rouge", "She wears a red dress"),
    ("Nous cuisinons le dîner ensemble", "We cook dinner together"),
    ("Il conduit une voiture bleue", "He drives a blue car"),
    ("Ils visitent souvent des musées", "They visit museums often"),
    ("Le restaurant sert une délicieuse cuisine", "The restaurant serves delicious food"),
    ("Elle étudie les mathématiques à l'université", "She studies mathematics at university"),
    ("Nous regardons des films le vendredi", "We watch movies on Fridays"),
    ("Il écoute de la musique en faisant du jogging", "He listens to music while jogging"),
    ("Ils voyagent autour du monde", "They travel around the world"),
    ("Le livre est sur la table", "The book is on the table"),
    ("Elle danse avec grâce", "She dances gracefully"),
    ("Nous célébrons les anniversaires avec un gâteau", "We celebrate birthdays with cake"),
    ("Il travaille dur tous les jours", "He works hard every day"),
    ("Ils parlent différentes langues", "They speak different languages"),
    ("Les fleurs fleurissent au printemps", "The flowers bloom in spring"),
    ("Elle écrit de la poésie pendant son temps libre", "She writes poetry in her free time"),
    ("Nous apprenons quelque chose de nouveau chaque jour", "We learn something new every day"),
    ("Le chien aboie bruyamment", "The dog barks loudly"),
    ("Il chante magnifiquement", "He sings beautifully"),
    ("Ils nagent dans la piscine", "They swim in the pool"),
    ("Les oiseaux gazouillent le matin", "The birds chirp in the morning"),
    ("Elle enseigne l'anglais à l'école", "She teaches English at school"),
    ("Nous prenons le petit déjeuner ensemble", "We eat breakfast together"),
    ("Il peint des paysages", "He paints landscapes"),
    ("Ils rient de la blague", "They laugh at the joke"),
    ("L'horloge tic-tac bruyamment", "The clock ticks loudly"),
    ("Elle court dans le parc", "She runs in the park"),
    ("Nous voyageons en train", "We travel by train"),
    ("Il écrit une lettre", "He writes a letter"),
    ("Ils lisent des livres à la bibliothèque", "They read books at the library"),
    ("Le bébé pleure", "The baby cries"),
    ("Elle étudie dur pour les examens", "She studies hard for exams"),
    ("Nous plantons des fleurs dans le jardin", "We plant flowers in the garden"),
    ("Il répare la voiture", "He fixes the car"),
    ("Ils boivent du café le matin", "They drink coffee in the morning"),
    ("Le soleil se couche le soir", "The sun sets in the evening"),
    ("Elle danse à la fête", "She dances at the party"),
    ("Nous jouons de la musique au concert", "We play music at the concert"),
    ("Il cuisine le dîner pour sa famille", "He cooks dinner for his family"),
    ("Ils étudient la grammaire française", "They study French grammar"),
    ("La pluie tombe doucement", "The rain falls gently"),
    ("Elle chante une chanson", "She sings a song"),
    ("Nous regardons un film ensemble", "We watch a movie together"),
    ("Il dort profondément", "He sleeps deeply"),
    ("Ils voyagent à Paris", "They travel to Paris"),
    ("Les enfants jouent dans le parc", "The children play in the park"),
    ("Elle se promène le long de la plage", "She walks along the beach"),
    ("Nous parlons au téléphone", "We talk on the phone"),
    ("Il attend le bus", "He waits for the bus"),
    ("Ils visitent la tour Eiffel", "They visit the Eiffel Tower"),
    ("Les étoiles scintillent la nuit", "The stars twinkle at night"),
    ("Elle rêve de voler", "She dreams of flying"),
    ("Nous travaillons au bureau", "We work in the office"),
    ("Il étudie l'histoire", "He studies history"),
    ("Ils écoutent la radio", "They listen to the radio"),
    ("Le vent souffle doucement", "The wind blows gently"),
    ("Elle nage dans l'océan", "She swims in the ocean"),
    ("Nous dansons au mariage", "We dance at the wedding"),
    ("Il gravit la montagne", "He climbs the mountain"),
    ("Ils font de la randonnée dans la forêt", "They hike in the forest"),
    ("Le chat miaule bruyamment", "The cat meows loudly"),
    ("Elle peint un tableau", "She paints a picture"),
    ("Nous construisons un château de sable", "We build a sandcastle"),
    ("Il chante dans le chœur", "He sings in the choir")
]

In [ ]:
dataset = FrEngDataset(french_to_english)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=10, dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

In [ ]:
encoder = Encoder(dataset.fr_vocab.n_words, 256).to(device)
decoder = AttnDecoder(256, dataset.eng_vocab.n_words, max_length=dataset.fr_tokens.shape[1], dropout_p=0.1).to(device)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=dataset.fr_tokens.shape[1]):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0
    correct_sequences = 0
    total_sequences = 0

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[dataset.eng_vocab.word2index["<SOS>"]]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == dataset.eng_vocab.word2index["<EOS>"]:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    total_sequences += 1
    if decoder_input.item() == target_tensor[di].item():
        correct_sequences += 1

    return loss.item() / target_length, correct_sequences, total_sequences

def evaluate(input_tensor, target_tensor, encoder, decoder, criterion, max_length=dataset.fr_tokens.shape[1]):
    with torch.no_grad():
        input_tensor = input_tensor.squeeze(0)
        target_tensor = target_tensor.squeeze(0)

        if input_tensor.size(0) == 0 or target_tensor.size(0) == 0:
            return 0, 0, 0

        encoder_hidden = encoder.initHidden()
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        eval_loss = 0
        correct_tokens = 0
        total_tokens = 0

        decoder_input = torch.tensor([[dataset.eng_vocab.word2index["<SOS>"]]], device=device)
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            eval_loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            if decoder_input.item() == dataset.eng_vocab.word2index["<EOS>"]:
                break

            total_tokens += 1
            if decoder_input.item() == target_tensor[di].item():
                correct_tokens += 1

        eval_loss /= target_length
        accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0

        return eval_loss.item(), accuracy, correct_tokens, total_tokens

def translate(input_tensor, encoder, decoder, max_length=dataset.fr_tokens.shape[1]):
    with torch.no_grad():
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[dataset.eng_vocab.word2index["<SOS>"]]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            if topi.item() == dataset.eng_vocab.word2index["<EOS>"]:
                break
            else:
                decoded_words.append(topi.item())

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
# Training and evaluation loop
n_epochs = 110
for epoch in range(n_epochs):
    total_loss = 0
    total_correct_sequences = 0
    total_sequences = 0
    val_total_loss = 0
    val_total_correct_tokens = 0
    val_total_tokens = 0

    for i, (input_tensor, target_tensor) in enumerate(dataloader):
        input_tensor, target_tensor = input_tensor.to(device), target_tensor.to(device)
        input_tensor = input_tensor.squeeze(0)
        target_tensor = target_tensor.squeeze(0)

        if input_tensor.size(0) == 0 or target_tensor.size(0) == 0:  # Skip empty tensors
            continue

        loss, correct_sequences, total_sequences = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=dataset.fr_tokens.shape[1])
        total_loss += loss
        total_correct_sequences += correct_sequences
        total_sequences += total_sequences

    with torch.no_grad():
        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            input_tensor, target_tensor = input_tensor.to(device), target_tensor.to(device)
            input_tensor = input_tensor.squeeze(0)
            target_tensor = target_tensor.squeeze(0)

            if input_tensor.size(0) == 0 or target_tensor.size(0) == 0:  # Skip empty tensors
                continue

            eval_loss, accuracy, correct_tokens, total_tokens = evaluate(input_tensor, target_tensor, encoder, decoder, criterion, max_length=dataset.fr_tokens.shape[1])
            val_total_loss += eval_loss
            val_total_correct_tokens += correct_tokens
            val_total_tokens += total_tokens

    # Calculate training and validation losses and accuracies
    train_loss = total_loss / len(dataloader)
    val_loss = val_total_loss / len(dataloader)
    train_accuracy = total_correct_sequences / len(dataset)  # Update training accuracy calculation
    val_accuracy = val_total_correct_tokens / val_total_tokens if val_total_tokens > 0 else 0

    if epoch % 10 == 0:
        # Print epoch-wise training and validation results
        print(f'Epoch {epoch}, Training Loss: {train_loss}, Training Accuracy: {train_accuracy}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Calculate overall evaluation results
overall_val_loss = val_total_loss / len(dataloader)
overall_val_accuracy = val_total_correct_tokens / val_total_tokens if val_total_tokens > 0 else 0
overall_train_loss = total_loss / len(dataloader)
overall_train_accuracy = total_correct_sequences / len(dataset)


# Printing prediction examples after training
print("\nPrediction Examples:")

n_examples = 10
example_count = 0

for i in range(len(dataset)):
    fr_tensor, eng_tensor = dataset[i]
    fr_tensor = fr_tensor.to(device)
    eng_tensor = eng_tensor.to(device)
    predicted_indices = translate(fr_tensor, encoder, decoder)

    predicted_string = ' '.join([dataset.eng_vocab.index2word[index] for index in predicted_indices if index not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])])
    target_string = ' '.join([dataset.eng_vocab.index2word[index.item()] for index in eng_tensor if index.item() not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])])
    fr_string = ' '.join([dataset.fr_vocab.index2word[index.item()] for index in fr_tensor if index.item() not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])])

    if example_count < n_examples:
        print(f'Example {example_count + 1}, Input: {fr_string}, Target: {target_string}, Predicted: {predicted_string}')
        example_count += 1

    if example_count >= n_examples:
        break

print("\nOverall Evaluation Results:")
print(f'Overall Training Loss: {overall_train_loss}, Overall Training Accuracy: {overall_train_accuracy}, Overall Validation Loss: {overall_val_loss}, Overall Validation Accuracy: {overall_val_accuracy}')


Epoch 0, Training Loss: 2.8079776172170052, Training Accuracy: 0.42857142857142855, Validation Loss: 1.535738965133568, Validation Accuracy: 0.3392857142857143
Epoch 10, Training Loss: 1.9125605675393194, Training Accuracy: 0.24675324675324675, Validation Loss: 1.6683259180613927, Validation Accuracy: 0.3198757763975155
Epoch 20, Training Loss: 1.774206538565058, Training Accuracy: 0.35064935064935066, Validation Loss: 1.586235748482989, Validation Accuracy: 0.33787465940054495
Epoch 30, Training Loss: 1.4489535824389, Training Accuracy: 0.36363636363636365, Validation Loss: 1.294970581283817, Validation Accuracy: 0.47354497354497355
Epoch 40, Training Loss: 0.9910576174957583, Training Accuracy: 0.7402597402597403, Validation Loss: 0.84356349242198, Validation Accuracy: 0.7238605898123325
Epoch 50, Training Loss: 0.4863692207253856, Training Accuracy: 0.922077922077922, Validation Loss: 0.3798231397356306, Validation Accuracy: 0.916267942583732
Epoch 60, Training Loss: 0.1621381762567

In [ ]:
english_sentences = [
"I am cold",
"You are tired",
"He is hungry",
"She is happy",
"We are friends",
"They are students",
"The cat is sleeping",
"The sun is shining",
"We love music",
"He enjoys reading books"
]

In [ ]:
# Function to prepare the sentence for translation
def prepare_for_translation(sentence, vocab):
    tokens = [vocab.word2index["<SOS>"]] + [vocab.word2index.get(word, vocab.word2index["<PAD>"]) for word in sentence.split(' ')] + [vocab.word2index["<EOS>"]]
    max_length = dataset.eng_tokens.shape[1]
    if len(tokens) < max_length:
        tokens += [vocab.word2index["<PAD>"]] * (max_length - len(tokens))
    return torch.tensor(tokens, dtype=torch.long, device=device).view(-1, 1)

# Function to translate English sentences to French using the trained model
def translate_sentence(sentence, encoder, decoder, eng_vocab, fr_vocab):
    input_tensor = prepare_for_translation(sentence, eng_vocab)
    translated_indices = translate(input_tensor, encoder, decoder)
    translated_words = [fr_vocab.index2word[index] for index in translated_indices if index not in (fr_vocab.word2index['<SOS>'], fr_vocab.word2index['<EOS>'], fr_vocab.word2index['<PAD>'])]
    return ' '.join(translated_words)

# Translating example sentences
for sentence in english_sentences:
    translation = translate_sentence(sentence, encoder, decoder, dataset.eng_vocab, dataset.fr_vocab)
    print(f'English Sentence: {sentence}\nFrench Translation: {translation}\n')

English Sentence: I am cold
French Translation: J'ai froid Tu

English Sentence: You are tired
French Translation: a souvent promène

English Sentence: He is hungry
French Translation: est faim heureuse

English Sentence: She is happy
French Translation: es fatigué Il

English Sentence: We are friends
French Translation: Nous fatigué sommes

English Sentence: They are students
French Translation: amis fatigué Ils

English Sentence: The cat is sleeping
French Translation: amis long mathématiques écoute plage

English Sentence: The sun is shining
French Translation: amis avec grâce écoute plage

English Sentence: We love music
French Translation: Nous soleil brille sa

English Sentence: He enjoys reading books
French Translation: amis nuit gâteau écoute rêve

